In [1]:
import tensorflow as tf
import numpy as np
import os
import time

### IMPORT DATA

In [2]:
text = open('data/shakespeare.txt', 'rb').read().decode(encoding='utf-8')
print(f'Length: {len(text)} characters')

Length: 1115394 characters


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### PREPROCESSING
TEXT UNIQUE

In [4]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


VECTORISER LE TEXT

In [5]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### TACHE DE PREDICTION

In [7]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [8]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

CONVERTIR LES CARACTERES EN SEQUENCE

In [10]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [13]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


SEQUENCE

In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

split_input_target(list('Tensorflow'))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [17]:
dataset = sequences.map(split_input_target)

In [18]:
for input_example, target_example in dataset.take(1):
    print(f"Input: {text_from_ids(input_example).numpy()}")
    print(f"Target: {text_from_ids(target_example).numpy()}")

Input: b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


CREER DES LOTS DE FORMATION (melanger et regrouper en lots)

In [19]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

### CONSTRUIRE LE MODELE

In [20]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units,
            return_sequences=True,
            return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)
        if return_state:
            return x, states
        else:
            return x

In [35]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

ESSAYER LE MODELE

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 66)


In [37]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [38]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([20,  6, 47, 13, 35,  1, 50, 46, 19, 34, 39, 55, 27, 46, 41, 50, 38,
       27, 35, 45, 46, 12, 47, 43, 46, 21, 18, 29,  4, 64, 54, 60, 18, 12,
       41, 36, 44, 41, 44, 54, 54,  0, 29, 64, 60, 18, 46, 52, 51, 43, 52,
       34, 29,  3,  3, 38,  1, 21, 43, 57, 44, 10, 27,  6, 41, 29,  3, 56,
       64, 32, 35, 15, 33, 33, 12, 20,  6, 40,  6, 36, 20, 18, 41, 21,  5,
       55, 57,  2,  2, 62, 60, 48, 59, 62, 25,  3,  5,  8, 15,  0],
      dtype=int64)

In [39]:
print(f'Input:\n{text_from_ids(input_example_batch[0]).numpy()}')
print()
print(f"Next:\n{text_from_ids(sampled_indices).numpy()}")

Input:
b"rwick, why hast thou withdrawn thyself?\nThy brother's blood the thirsty earth hath drunk,\nBroach'd w"

Next:
b"G'h?V\nkgFUZpNgbkYNVfg;hdgHEP$youE;bWebeoo[UNK]PyuEgmldmUP!!Y\nHdre3N'bP!qySVBTT;G'a'WGEbH&pr  wuitwL!&-B[UNK]"


### FORMER LE MODEL
ATTACHER UN OPTIMIZER ET UNE FONCTION DE PERTE

In [40]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(f"Prediction shape: {example_batch_predictions.shape}")
print(f"Mean loss: {example_batch_mean_loss}")

Prediction shape: (64, 100, 66)
Mean loss: 4.190152168273926


EXPONENTIELLE DE LA PERTE

In [41]:
tf.exp(example_batch_mean_loss).numpy() # ~ a la taille du cobulaire

66.03284

In [42]:
model.compile(optimizer='adam', loss=loss)

CONFIGURER LES POINTS DE CONTROLE

In [43]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

EXECUTER LA FORMATION

In [44]:
EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 284s 2s/step - loss: 2.7391
Epoch 2/20
172/172 [==============================] - 291s 2s/step - loss: 2.0000
Epoch 3/20
172/172 [==============================] - 293s 2s/step - loss: 1.7279
Epoch 4/20
172/172 [==============================] - 294s 2s/step - loss: 1.5629
Epoch 5/20
172/172 [==============================] - 296s 2s/step - loss: 1.4609
Epoch 6/20
172/172 [==============================] - 297s 2s/step - loss: 1.3907
Epoch 7/20
172/172 [==============================] - 300s 2s/step - loss: 1.3378
Epoch 8/20
172/172 [==============================] - 299s 2s/step - loss: 1.2924
Epoch 9/20
172/172 [==============================] - 297s 2s/step - loss: 1.2525
Epoch 10/20
172/172 [==============================] - 295s 2s/step - loss: 1.2134
Epoch 11/20
172/172 [==============================] - 295s 2s/step - loss: 1.1745
Epoch 12/20
172/172 [==============================] - 296s 2s/step - loss: 1.1342
Epoch 13/20
1

### GENERATE TEXT

In [47]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars
        
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            values=[-float('inf')] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())]
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        # CONVERT STRING TO TOKEN IDS
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()
        # RUN MODEL
        predicted_logits, states = self.model(inputs=input_ids, states=states, return_state=True)
        # USE THE LAST PREDICTION
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # GENERATE TOKEN IDS
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
        # IDS TO CHARACTERS
        predicted_chars = self.chars_from_ids(predicted_ids)
        # RETURN THE CHARACTERS AND MODEL STATE
        return predicted_chars, states

In [48]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

### EXECUTER EN BOUCLE POUR GENERER DU TEXTE
<b>Le model sait:</b> <br>
-Quand capitaliser <br>
-Faire des paragraphes et imiter un vocabulaire de type Shakespeare<br>
<b>Le model n'a pas encore appris a former des phrase coherents</b> <br>
(A cause du temps d'entrainement)

In [49]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO'])
result = [next_char]
for n in range(1000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)
    
result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time: ', end - start)

ROMEO:
The time be wit'st up:' Tybalt, great Capebit
To see the worfest larg and a pleasing royalty,
And her with slaughter'd Rivensm to Rome,
And I'll break his throat; under your highness
Will wot that shall rent suffice. I pray thee, Capulet,--

CORIOLANUS:
What he will sop't?

MENENIUS:
The seasols pale and sorrow, then again with blood,
With thought their news abwed: if I
Had seen the letter, his conveyance enemy:
As cull all esses called as my brother,--
Love bold lords, my Lord of Buckingham and
My wife in his bed, when you weep in
advise.

ISABELLA:
We have swoon it is the best.

WARWICK:
Peace, Petruchio, sweet and players say,
As King of England's gardan. Where's resort?

Third Servingman:
Aating with like thing in extremity.

WARWICK:
Exen in his neisting stars in Marsius his
frowning: what's enemies? Oh, Montague,
Speak supstantion, being that trumpet'st sounded;
And thou, Harry Herbiora, I will dance.
Love given and Clarence; therefore I can deserve her
Doke that Forrest! 

### SAVE MODEL

In [50]:
tf.saved_model.save(one_step_model, 'one_step')

INFO:tensorflow:Assets written to: one_step\assets


INFO:tensorflow:Assets written to: one_step\assets
